`런타임 > 런타임 유형 변경 > T4 GPU` 이 과정을 반드시 먼저 수행하고 하셔야 GPU를 사용할 수 있습니다.

In [ ]:
!pip install PyPDF2 datasets sentence-transformers==3.4.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import requests
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from openai import OpenAI
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, losses, InputExample
from sentence_transformers.evaluation import InformationRetrievalEvaluator
import torch
from sklearn.metrics.pairwise import cosine_similarity
import PyPDF2

In [ ]:
# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = "여러분의 Key 값"

# 1. PDF 파일 다운로드
urls = [
   "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch06/ict_japan_2024.pdf",
   "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch06/ict_usa_2024.pdf"
]

In [ ]:
for url in urls:
   filename = url.split("/")[-1]
   response = requests.get(url)
   with open(filename, "wb") as f:
       f.write(response.content)
   print(f"{filename} 다운로드 완료")

ict_japan_2024.pdf 다운로드 완료
ict_usa_2024.pdf 다운로드 완료


In [ ]:
def extract_text_from_pdf(pdf_path):
   """PDF 파일에서 텍스트를 추출하는 함수"""
   text_chunks = []
   with open(pdf_path, 'rb') as file:
       pdf_reader = PyPDF2.PdfReader(file)
       for page_num in range(len(pdf_reader.pages)):
           page = pdf_reader.pages[page_num]
           text = page.extract_text()
           # 페이지 단위로 청크 생성
           if text.strip():
               text = text.strip()
               # 문서 길이가 10자 초과인 경우만 추가
               if len(text) > 10:
                   text_chunks.append(text)
   return text_chunks

# 미국 ICT 동향 (학습 데이터)
train_corpus = extract_text_from_pdf('ict_usa_2024.pdf')
print(f'학습 데이터 문서 개수: {len(train_corpus)}')

# 일본 ICT 동향 (검증 데이터)
val_corpus = extract_text_from_pdf('ict_japan_2024.pdf')
print(f'검증 데이터 문서 개수: {len(val_corpus)}')

학습 데이터 문서 개수: 26
검증 데이터 문서 개수: 27


In [ ]:
print('10번 문서:', train_corpus[10])

10번 문서: 13 Ⅰ. ICT 국가 산업 현황
 4.ICT 주요 법령 및 규제
  ② 반도체 과학법 (CHIPS and Science Act)
 반도체 ·전자 기업, $1,660 억 규모 투자 유치 
• 조 바이든 (Joe Biden) 미국 대통령은 2022년 7월 ‘반도체 과학법 (CHIPS and Science Act)’을 승인함  
• 반도체 과학법은 미국의 경쟁력을 강화하고 , 미국의 공급망을 탄력적으로 구축해 국가 안보를 
공고히 하며 국가의 주요 기술에 대한 접근을 지원하는 것을 목표로 함. 법률 제정으로 미국 내 
반도체 생산 제조사 관련 자본 투자는 25%의 세액 공제 혜택이 제공됨
• 미국 백악관은 2023년 8월, 반도체 과학법이 서명된 지 1년 만에 반도체와 전자 관련 기업들이 
1,660 억 달러(221조6,100 억 원)의 투자를 유치했다고 발표함 . 바이든 행정부의 집권 이후 기업들은 
미국 내 반도체와 전자 분야 투자에 총 2,310 억 달러(약 308조 3,850 억 원) 이상의 투자를 약속함
[표 8] 반도체 과학법 주요 이정표 및 진척 현황
주요 이정표 진척 현황 (23년 8월 기준)
미국 반도체 제조 
지원‣ 상무부는 CHIPS 통과 6개월 만에 해당 법에서 제공하는 390억 달러 반도체 제조 
인센티브에 대한 첫 번째 자금 조달 기회 시작
‣ 상무부는 42개 주에서 CHIPS 자금 지원에 관심 있는 460개 기업으로부터 소개서 접수
‣ 상무부는 CHIPS 인센티브 프로그램 관련 140명 이상의 인력으로 구성된 ‘칩스 포 
아메리카 (CHIPS for America)’ 팀 구성
‣ 재무부는 투자에 대한 25% 세액 공제 관련 지침 제공 위해 규칙 제안 발표
국가 안보를 
보호하고 동맹국 및 
파트너와 협력‣ 국무부는 국제 기술 보안 및 혁신 기금 시행 계획 발표
‣ 국방부와 상무부는 CHIPS 투자를 통한 안보 관련 반도체 제조를 위해 협력 확대 협의
‣ CHIPS 를 시행하면서 상무부는 여러 파트너 및 동맹국과 긴밀한 접촉

In [ ]:
# OpenAI 클라이언트 초기화
client = OpenAI()

# 3. 각 문서에 대한 질문 생성 (OpenAI API 사용)
def generate_queries(corpus, num_questions_per_chunk=2):
    all_queries = []
    all_positive_docs = []

    # 기본 프롬프트 템플릿 설정
    prompt_template = """\
    다음은 참고할 내용입니다.

    ---------------------
    {context_str}
    ---------------------

    위 내용을 바탕으로 낼 수 있는 질문을 {num_questions_per_chunk}개 만들어주세요.
    질문만 작성하고 실제 정답이나 보기 등은 작성하지 않습니다.

    해당 질문은 본문을 볼 수 없다고 가정합니다.
    따라서 '위 본문을 바탕으로~' 라는 식의 질문은 할 수 없습니다.

    질문은 아래와 같은 형식으로 번호를 나열하여 생성하십시오.

    1. (질문)
    2. (질문)
    """

    # corpus의 각 문서에 대해 반복 실행
    for text in tqdm(corpus):
        # 현재 문서에 대한 프롬프트 생성
        messages = [
            {"role": "system", "content": "You are a helpful assistant that generates questions based on provided content."},
            {"role": "user", "content": prompt_template.format(
                context_str=text,
                num_questions_per_chunk=num_questions_per_chunk
            )}
        ]

        # GPT 모델을 사용해 질문 생성
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0.7,
        )

        # 응답을 줄바꿈을 기준으로 분리하여 개별 질문으로 만듦
        result = response.choices[0].message.content.strip().split("\n")

        # 질문 형식 정리
        questions = []
        for line in result:
            if line.strip():
                parts = line.strip().split('. ', 1)
                if len(parts) > 1:
                    questions.append(parts[1])
                else:
                    questions.append(parts[0])

        # 빈 질문 제거
        questions = [q for q in questions if len(q) > 0]

        # 각 질문에 대해 문서 매칭 및 저장
        for question in questions:
            all_queries.append(question)
            all_positive_docs.append(text)

    return all_queries, all_positive_docs

In [ ]:
# 학습 데이터 질문 생성
train_queries, train_positive_docs = generate_queries(train_corpus)
print(f'생성된 학습용 질문 개수: {len(train_queries)}')

# 검증 데이터 질문 생성
val_queries, val_positive_docs = generate_queries(val_corpus)
print(f'생성된 검증용 질문 개수: {len(val_queries)}')

# 4. 학습 데이터 준비
train_examples = []
for query, doc in zip(train_queries, train_positive_docs):
    example = InputExample(texts=[query, doc])
    train_examples.append(example)

  0%|          | 0/26 [00:00<?, ?it/s]

생성된 학습용 질문 개수: 52


  0%|          | 0/27 [00:00<?, ?it/s]

생성된 검증용 질문 개수: 54


In [ ]:
# 첫번째 샘플
train_examples[0].texts

['미국의 빅테크 기업들이 집중하고 있는 인공지능 기술의 한 분야는 무엇인가요?',
 'Ⅰ ICT국가산업현황  4\n(*) SUMMARY\n1. 국가 개황\n2. ICT 정부기구\n3. ICT 주요정책\n4. ICT 주요법령및규제\n5. ICT 주요기업\n6. 한국 협력 및 국내기업 진출사례\nⅡ ICT이슈Top 10  16\n(*) SUMMARY\n① 미국 빅테크 기업, 인공지능 챗봇 개발에 주력\n② 미국, 일본과 양자컴퓨팅 개발 협력\n③ 미국, 우주 클라우드 컴퓨팅 시장 주도\n④ 미국, 드론 배송 도입 활발\n⑤ 미국, 긍정적인 의료 AI 인식 바탕으로 연구 활발\n⑥ 미국, 반도체 산업 활성화에 박차\n⑦ 미국, 기술 교류를 위한 국가 간 협력 활발\n⑧ 미국, 사이버 보안 대응 강화\n⑨ 미국, 6G 주도권 확보 위한 연구 추진\n⑩ 미 국방부 , 디지털 트윈 기술 도입 확대\n※ 참고문헌']

In [ ]:
# 두번째 샘플
train_examples[1].texts

['미국이 일본과 협력하여 개발을 추진하고 있는 첨단 기술은 무엇인가요?',
 'Ⅰ ICT국가산업현황  4\n(*) SUMMARY\n1. 국가 개황\n2. ICT 정부기구\n3. ICT 주요정책\n4. ICT 주요법령및규제\n5. ICT 주요기업\n6. 한국 협력 및 국내기업 진출사례\nⅡ ICT이슈Top 10  16\n(*) SUMMARY\n① 미국 빅테크 기업, 인공지능 챗봇 개발에 주력\n② 미국, 일본과 양자컴퓨팅 개발 협력\n③ 미국, 우주 클라우드 컴퓨팅 시장 주도\n④ 미국, 드론 배송 도입 활발\n⑤ 미국, 긍정적인 의료 AI 인식 바탕으로 연구 활발\n⑥ 미국, 반도체 산업 활성화에 박차\n⑦ 미국, 기술 교류를 위한 국가 간 협력 활발\n⑧ 미국, 사이버 보안 대응 강화\n⑨ 미국, 6G 주도권 확보 위한 연구 추진\n⑩ 미 국방부 , 디지털 트윈 기술 도입 확대\n※ 참고문헌']

In [ ]:
# 세번째 샘플
train_examples[2].texts

['ICT 국가 산업 현황에서 언급된 주요 정부기구에는 어떤 것들이 있나요?',
 'Ⅰ ICT 국가 산업 현황                   4\n   (*) SUMMARY\n   1. 국가 개황\n   2. ICT 정부기구\n   3. ICT 주요 정책\n   4. ICT 주요 법령 및 규제\n   5. ICT 주요기업\n   6. 한국 협력 및 국내기업 진출사례']

In [ ]:
# 네번째 샘플
train_examples[3].texts

['한국과의 협력 사례 및 국내 기업의 진출 사례에는 어떤 것들이 포함되어 있나요?',
 'Ⅰ ICT 국가 산업 현황                   4\n   (*) SUMMARY\n   1. 국가 개황\n   2. ICT 정부기구\n   3. ICT 주요 정책\n   4. ICT 주요 법령 및 규제\n   5. ICT 주요기업\n   6. 한국 협력 및 국내기업 진출사례']

In [ ]:
queries = [sample.texts[0] for sample in train_examples]

In [ ]:
len(queries)

52

In [ ]:
train_examples[0].texts

['미국의 빅테크 기업들이 집중하고 있는 인공지능 기술의 한 분야는 무엇인가요?',
 'Ⅰ ICT국가산업현황  4\n(*) SUMMARY\n1. 국가 개황\n2. ICT 정부기구\n3. ICT 주요정책\n4. ICT 주요법령및규제\n5. ICT 주요기업\n6. 한국 협력 및 국내기업 진출사례\nⅡ ICT이슈Top 10  16\n(*) SUMMARY\n① 미국 빅테크 기업, 인공지능 챗봇 개발에 주력\n② 미국, 일본과 양자컴퓨팅 개발 협력\n③ 미국, 우주 클라우드 컴퓨팅 시장 주도\n④ 미국, 드론 배송 도입 활발\n⑤ 미국, 긍정적인 의료 AI 인식 바탕으로 연구 활발\n⑥ 미국, 반도체 산업 활성화에 박차\n⑦ 미국, 기술 교류를 위한 국가 간 협력 활발\n⑧ 미국, 사이버 보안 대응 강화\n⑨ 미국, 6G 주도권 확보 위한 연구 추진\n⑩ 미 국방부 , 디지털 트윈 기술 도입 확대\n※ 참고문헌']

In [ ]:
train_examples[1].texts

['미국이 일본과 협력하여 개발을 추진하고 있는 첨단 기술은 무엇인가요?',
 'Ⅰ ICT국가산업현황  4\n(*) SUMMARY\n1. 국가 개황\n2. ICT 정부기구\n3. ICT 주요정책\n4. ICT 주요법령및규제\n5. ICT 주요기업\n6. 한국 협력 및 국내기업 진출사례\nⅡ ICT이슈Top 10  16\n(*) SUMMARY\n① 미국 빅테크 기업, 인공지능 챗봇 개발에 주력\n② 미국, 일본과 양자컴퓨팅 개발 협력\n③ 미국, 우주 클라우드 컴퓨팅 시장 주도\n④ 미국, 드론 배송 도입 활발\n⑤ 미국, 긍정적인 의료 AI 인식 바탕으로 연구 활발\n⑥ 미국, 반도체 산업 활성화에 박차\n⑦ 미국, 기술 교류를 위한 국가 간 협력 활발\n⑧ 미국, 사이버 보안 대응 강화\n⑨ 미국, 6G 주도권 확보 위한 연구 추진\n⑩ 미 국방부 , 디지털 트윈 기술 도입 확대\n※ 참고문헌']

In [ ]:
train_examples[2].texts

['ICT 국가 산업 현황에서 언급된 주요 정부기구에는 어떤 것들이 있나요?',
 'Ⅰ ICT 국가 산업 현황                   4\n   (*) SUMMARY\n   1. 국가 개황\n   2. ICT 정부기구\n   3. ICT 주요 정책\n   4. ICT 주요 법령 및 규제\n   5. ICT 주요기업\n   6. 한국 협력 및 국내기업 진출사례']

In [ ]:
# 임의의 25 샘플
print(train_examples[25].texts)  # 질문과 문서 내용이 리스트로 출력됨

['최근 몇 년간 한국의 ICT 기업들이 미국 시장에 진출한 사례는 무엇인가요?', "15 Ⅰ. ICT 국가 산업 현황\n 6.한국 협력 및 국내기업 진출사례\n 한국-미국 FTA 체결 여부\n• 2012년 3월 한-미 FTA 발효됨\n• 2019년 1월 한-미 FTA 개정 의정서 발효, 투자자 -국가분쟁해결제도 (ISDS) 와 수출기업에 부담이 \n된 무역구제 절차 개선됨\n• 2022년 한-미 FTA 10주년을 기념해 정부·국회 대표단이 워싱턴 D.C(Washington DC), \n미시간 (Michigan), 뉴욕(New York) 등을 방문함\n 한국-미국 ICT 기관 협력 사례\n• 과기정통부와 미국 기관 간 ICT 협력 사례가 주목됨 \n 한국-미국 ICT 기업 진출 사례\n• 다양한 산업과 규모의 한국 기업들이 미국 시장 진출을 위해 노력하고 있음 \n[표 10] 한국-미국 협력 현황\n구분 날짜 내용\nFTA 체결여부\n(발효)12.03 ‣ 한-미 FTA 발효\n19.01 ‣ 한-미 FTA 개정 의정서 발효\n22.03 ‣ 한-미 FTA 10주년 기념해 정부·국회 대표단 방미 일정 진행\n정부23.01‣ 경북도 , 산타클라라 한인상공회의소와 '실리콘밸리 스타트업 아카데미 \n경북 MOU' 개최\n23.01 ‣ 광주시 , AI 스타트업 미국 진출 지원\n23.01‣ 화성시 , 실리콘밸리 산타클라라 한인상공회의소와 ‘스타트업 기업의 \n글로벌 진출 지원 업무협약 ’ 체결\n23.07 ‣ 과기정통부 , 한·미 WRC 협력회의 개최\n24.01 ‣ 과기정통부 , 미국 CES 2024서 ‘디지털 청년 인재 포럼’ 개최\n기업23.01 ‣ 바이든 대통령 , 한화 솔루션 미국 조지아주 ‘솔라 허브’ 투자 결정 환영\n23.01 ‣ 필워크 , 미국 법인 설립 및 앱 서비스 론칭으로 미국 시장 본격 진출\n23.08 ‣ 현대차그룹 모셔널 , ‘아이오닉 5 로보 택시’ 로스앤젤레스로 서비스 확장\n23.08 ‣ 토마토시스템 , 미국 원격 진료 시장 진출\n24.0

In [ ]:
print("문서:")
print(train_examples[25].texts[1])  # 두 번째 요소는 문서 내용
print('--' * 50)
print("문서로부터 생성한 질문:")
print(train_examples[25].texts[0])  # 첫 번째 요소는 질문

문서:
15 Ⅰ. ICT 국가 산업 현황
 6.한국 협력 및 국내기업 진출사례
 한국-미국 FTA 체결 여부
• 2012년 3월 한-미 FTA 발효됨
• 2019년 1월 한-미 FTA 개정 의정서 발효, 투자자 -국가분쟁해결제도 (ISDS) 와 수출기업에 부담이 
된 무역구제 절차 개선됨
• 2022년 한-미 FTA 10주년을 기념해 정부·국회 대표단이 워싱턴 D.C(Washington DC), 
미시간 (Michigan), 뉴욕(New York) 등을 방문함
 한국-미국 ICT 기관 협력 사례
• 과기정통부와 미국 기관 간 ICT 협력 사례가 주목됨 
 한국-미국 ICT 기업 진출 사례
• 다양한 산업과 규모의 한국 기업들이 미국 시장 진출을 위해 노력하고 있음 
[표 10] 한국-미국 협력 현황
구분 날짜 내용
FTA 체결여부
(발효)12.03 ‣ 한-미 FTA 발효
19.01 ‣ 한-미 FTA 개정 의정서 발효
22.03 ‣ 한-미 FTA 10주년 기념해 정부·국회 대표단 방미 일정 진행
정부23.01‣ 경북도 , 산타클라라 한인상공회의소와 '실리콘밸리 스타트업 아카데미 
경북 MOU' 개최
23.01 ‣ 광주시 , AI 스타트업 미국 진출 지원
23.01‣ 화성시 , 실리콘밸리 산타클라라 한인상공회의소와 ‘스타트업 기업의 
글로벌 진출 지원 업무협약 ’ 체결
23.07 ‣ 과기정통부 , 한·미 WRC 협력회의 개최
24.01 ‣ 과기정통부 , 미국 CES 2024서 ‘디지털 청년 인재 포럼’ 개최
기업23.01 ‣ 바이든 대통령 , 한화 솔루션 미국 조지아주 ‘솔라 허브’ 투자 결정 환영
23.01 ‣ 필워크 , 미국 법인 설립 및 앱 서비스 론칭으로 미국 시장 본격 진출
23.08 ‣ 현대차그룹 모셔널 , ‘아이오닉 5 로보 택시’ 로스앤젤레스로 서비스 확장
23.08 ‣ 토마토시스템 , 미국 원격 진료 시장 진출
24.01 ‣ SK텔레콤 , 동물 AI 진단 솔루션 엑스칼리버 미국시장 본격 공략
24.02 ‣ 올거나이즈 , SOC-2 및 HIPPA 미

In [ ]:
# 5. 데이터 로더 생성
# 현재 Colab GPU의 한계로 매우 작은 배치 크기인 5를 선택했습니다.
# 일반적으로는 배치 크기가 클수록 성능이 좋습니다.
BATCH_SIZE = 4  # 배치 크기 조정
loader = DataLoader(train_examples, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# 6. 모델 및 손실 함수 설정
model_id = "BAAI/bge-m3"
model = SentenceTransformer(model_id)

# 손실 함수 정의: 모델이 검색어와 포지티브 문서는 가깝게, 네거티브 문서는 멀게 학습하도록 유도
loss = losses.MultipleNegativesRankingLoss(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
# 7. 검증 데이터 평가기 설정
# 검증 데이터셋 구성
val_dataset = {
    'queries': {},
    'corpus': {},
    'relevant_docs': {}
}

# 문서 ID를 먼저 생성
doc_ids = {}
for i, doc in enumerate(val_corpus):
    doc_id = f"d{i}"
    val_dataset['corpus'][doc_id] = doc
    doc_ids[doc] = doc_id

# 질문에 ID 부여하고 관련 문서 설정
for i, (query, doc) in enumerate(zip(val_queries, val_positive_docs)):
    query_id = f"q{i}"
    val_dataset['queries'][query_id] = query

    # 이 질문이 어떤 문서에서 왔는지 찾기
    doc_id = doc_ids[doc]

    # 관련 문서 설정
    if query_id not in val_dataset['relevant_docs']:
        val_dataset['relevant_docs'][query_id] = set()
    val_dataset['relevant_docs'][query_id].add(doc_id)

# 검증 데이터셋 설정: 평가를 위한 쿼리, 문서, 정답 문서 목록
dataset = val_dataset

# 검증 데이터셋에서 코퍼스(전체 문서), 쿼리, 그리고 각 쿼리와 관련된 문서 가져오기
corpus = dataset['corpus']  # 검색 대상 문서
queries = dataset['queries']  # 검색어(쿼리)
relevant_docs = dataset['relevant_docs']  # 각 쿼리와 관련된 문서 (포지티브)

# Information Retrieval 평가 도구 설정: 쿼리-문서 검색 성능 평가
evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [ ]:
dataset['corpus']

{'d0': 'Ⅰ ICT국가산업현황  4\n(*) SUMMARY\n1. 국가 개황\n2. ICT 정부기구\n3. ICT 주요정책\n4. ICT 주요법령및규제\n5. ICT 주요기업\n6. 한국 협력 및 국내기업 진출사례\nⅡ ICT이슈Top 10  16\n(*) SUMMARY\n① 일본, 아시아에서 두 번째로 큰 데이터센터 허브\n② 일본, 자체 개발 소프트웨어로 사이버보안 강화\n③ 일본, Web3 산업 성장 촉진 도모\n④ 일본, 정부 행정 업무에 생성형 AI 도입\n⑤ 일본, 첫 자체 제작 양자컴퓨터 공개\n⑥ 일본, 6G 기술 강화 위해 협력 및 규제 완화\n⑦ 일본, 레벨 4 자율주행 허용\n⑧ 일본, 생체인식 결제 도입 증가\n⑨ 일본, 행정 서비스 디지털화 노력\n⑩ 일본, 인재 부족으로 디지털 인력 강화에 힘써\n※ 참고문헌',
 'd1': 'Ⅰ ICT 국가 산업 현황                   4\n   (*) SUMMARY\n   1. 국가 개황\n   2. ICT 정부기구\n   3. ICT 주요 정책\n   4. ICT 주요 법령 및 규제\n   5. ICT 주요기업\n   6. 한국 협력 및 국내기업 진출사례',
 'd2': '5 Ⅰ. ICT 국가 산업 현황\n 1.국가 개황\n 일본, 글로벌 혁신지수 세계 40위\n• 일본의 인터넷 사용자 비중은 82.9% 이며 고정 광대역 가입자 비중은 36.0% 임\n• 일본 글로벌 혁신지수는 세계 13위임. ‘인프라 ’ 및 ‘지식 및 기술 생산’ 지표가 상대적으로 \n우위에 있으며 , ‘창조적 생산’이 상대적 열위를 보임\n 기시다 총리, 결제 활성화 정책에 초점\n• 2023년 일본 물가상승률은 3.1%로 41년에 최대폭을 기록함 . 일본은행은 2023년 12월 \n금융정책결정회의에서 단기금리 목표를 △0.1%로 동결하면서 마이너스 금리 종료를 보류함\n• 일본 기시다 후미오 (岸⽥⽂雄 ) 총리는 향후 경제 활성화를 위한 정책에 초점을 맞출 것임을 밝힘. \n그중에서도 감세 논의에 속도를 

In [ ]:
dataset['queries']

{'q0': '일본에서 아시아에서 두 번째로 큰 데이터센터 허브로 자리 잡은 이유는 무엇인가요?',
 'q1': '일본이 6G 기술 강화를 위해 어떤 협력 및 규제 완화 조치를 취하고 있나요?',
 'q2': 'ICT 정부기구의 주요 역할과 기능은 무엇인가요?',
 'q3': '한국과의 협력 및 국내기업의 ICT 산업 진출 사례는 어떤 것들이 있나요?',
 'q4': '2023년 일본의 글로벌 혁신지수 순위는 몇 위인가요?',
 'q5': '2023년 일본의 경제 활성화를 위해 기시다 후미오 총리가 초점을 맞추고 있는 정책은 무엇인가요?',
 'q6': '일본 총무성(MIC)이 담당하는 주요 분야는 무엇인가요?',
 'q7': '일본 총무성이 최근 발간한 로컬 5G 관련 가이드라인의 목적은 무엇인가요?',
 'q8': '일본 문부과학성(MEXT)은 교육 부문에서 어떤 디지털 전환(DX) 프로젝트를 추진하고 있나요?',
 'q9': '일본 문부과학성이 도도부현에 조성한 ICT 관련 기금 규모는 얼마이며, 이 기금은 어떤 목적을 위해 사용되나요?',
 'q10': '일본 경제산업성(METI)은 어떤 분야에서 국제 표준을 발표했으며, 그 목적은 무엇입니까?',
 'q11': '일본 경제산업성이 최근 시작한 자율주행 모빌리티 서비스 프로젝트는 어떤 부서와 협업하여 진행되었습니까?',
 'q12': '일본 디지털청의 발족 시기와 그 목표는 무엇인가요?',
 'q13': '일본 디지털청에서 최근 실행한 정책 중 하나로 스마트폰과 관련된 서비스는 무엇인가요?',
 'q14': "일본 개인정보보호위원회가 발표한 '생성형 AI 서비스 사용에 관한 지침'에 따르면 개인정보를 프롬프트로 입력할 때 주의해야 할 점은 무엇인가요?",
 'q15': '생성형 AI 서비스 제공 사업자가 개인정보를 기계 학습에 이용하지 않기 위해 어떤 확인이 필요한가요?',
 'q16': '일본 내각부가 AI 모델 성능 향상을 위해 추진하는 액션플랜의 주요 목표는 무엇인가요?',
 'q17': '일본 정

In [ ]:
dataset['relevant_docs']

{'q0': {'d0'},
 'q1': {'d0'},
 'q2': {'d1'},
 'q3': {'d1'},
 'q4': {'d2'},
 'q5': {'d2'},
 'q6': {'d3'},
 'q7': {'d3'},
 'q8': {'d4'},
 'q9': {'d4'},
 'q10': {'d5'},
 'q11': {'d5'},
 'q12': {'d6'},
 'q13': {'d6'},
 'q14': {'d7'},
 'q15': {'d7'},
 'q16': {'d8'},
 'q17': {'d8'},
 'q18': {'d9'},
 'q19': {'d9'},
 'q20': {'d10'},
 'q21': {'d10'},
 'q22': {'d11'},
 'q23': {'d11'},
 'q24': {'d12'},
 'q25': {'d12'},
 'q26': {'d13'},
 'q27': {'d13'},
 'q28': {'d14'},
 'q29': {'d14'},
 'q30': {'d15'},
 'q31': {'d15'},
 'q32': {'d16'},
 'q33': {'d16'},
 'q34': {'d17'},
 'q35': {'d17'},
 'q36': {'d18'},
 'q37': {'d18'},
 'q38': {'d19'},
 'q39': {'d19'},
 'q40': {'d20'},
 'q41': {'d20'},
 'q42': {'d21'},
 'q43': {'d21'},
 'q44': {'d22'},
 'q45': {'d22'},
 'q46': {'d23'},
 'q47': {'d23'},
 'q48': {'d24'},
 'q49': {'d24'},
 'q50': {'d25'},
 'q51': {'d25'},
 'q52': {'d26'},
 'q53': {'d26'}}

## 6. 실제 학습

In [ ]:
# 7. 모델 학습
EPOCHS = 2

In [ ]:
# W&B(WandB, Weights and Biases) 로깅 비활성화
# W&B는 학습 과정을 실시간으로 추적하고 시각화할 수 있는 도구입니다.
# 하지만 이 코드는 W&B 기능을 사용하지 않으므로, 이를 비활성화하여 로깅을 중단합니다.
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# 학습 초기에 학습률을 점진적으로 증가시키는 단계 수 설정
# 전체 학습 단계의 10%를 워밍업으로 사용
warmup_steps = int(len(loader) * EPOCHS * 0.1)

# 모델 학습
model.fit(
    train_objectives=[(loader, loss)],  # 학습 데이터 로더와 손실 함수 설정
    epochs=EPOCHS,  # 총 에포크 수
    warmup_steps=warmup_steps,  # 워밍업 단계
    output_path='exp_finetune',  # 학습된 모델 저장 경로
    show_progress_bar=True,  # 학습 진행률 표시 여부
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


In [ ]:
def evaluate_st(dataset, model_id, name, evaluator):
    """
    SentenceTransformer 모델의 검색 성능을 평가하는 함수
    """
    # 평가 결과를 저장할 디렉토리 생성
    os.makedirs('results', exist_ok=True)

    # 평가할 SentenceTransformer 모델 로드
    model = SentenceTransformer(model_id)

    # 모델 평가 수행
    result = evaluator(model)

    # 결과를 DataFrame으로 변환하고 CSV로 저장
    result_df = pd.DataFrame([result]) if isinstance(result, dict) else result
    output_path = f'results/Information-Retrieval_evaluation_{name}_results.csv'
    result_df.to_csv(output_path, index=False)

    return result

In [ ]:
# SentenceTransformer 평가 (추가 지표 계산)
finetuned_model_path = "exp_finetune"  # 파인튜닝된 모델 경로
evaluate_st(dataset=val_dataset, model_id=finetuned_model_path, name='finetuned', evaluator=evaluator)
print("파인튜닝 모델 평가 완료")

파인튜닝 모델 평가 완료


In [ ]:
# SentenceTransformer 평가 (추가 지표 계산)
original_model_path = "BAAI/bge-m3"  # 원본 모델 ID
evaluate_st(dataset=val_dataset, model_id=original_model_path, name='original', evaluator=evaluator)
print("원본 모델 평가 완료")

원본 모델 평가 완료


In [ ]:
# 결과 비교
df_st_original = pd.read_csv('results/Information-Retrieval_evaluation_original_results.csv')
df_st_finetuned = pd.read_csv('results/Information-Retrieval_evaluation_finetuned_results.csv')

df_st_original['model'] = 'bge-m3'
df_st_finetuned['model'] = 'fine_tuned'
df_st_all = pd.concat([df_st_original, df_st_finetuned])
df_st_all = df_st_all.set_index('model')

print("\n모델 성능 비교:")
df_st_all


모델 성능 비교:


,cosine_accuracy@1,cosine_accuracy@3,cosine_accuracy@5,cosine_accuracy@10,cosine_precision@1,cosine_precision@3,cosine_precision@5,cosine_precision@10,cosine_recall@1,cosine_recall@3,cosine_recall@5,cosine_recall@10,cosine_ndcg@10,cosine_mrr@10,cosine_map@100
model,,,,,,,,,,,,,,,
bge-m3,0.833333,0.944444,0.962963,1.0,0.833333,0.314815,0.192593,0.1,0.833333,0.944444,0.962963,1.0,0.910484,0.882253,0.882253
fine_tuned,0.907407,1.000000,1.000000,1.0,0.907407,0.333333,0.200000,0.1,0.907407,1.000000,1.000000,1.0,0.958553,0.944444,0.944444
